In [ ]:
!nvidia-smi

Mon Jan 17 09:33:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle

with open("datastore.pkl", "rb") as fl:
  datastore = pickle.load(fl)

In [ ]:
print(f"Total Lines: {len(datastore)}")

Total Lines: 54534


In [ ]:
datastore = [str(line) for line in datastore[:500]]

In [ ]:
print(f"Total Lines: {len(datastore)}")

Total Lines: 500


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(datastore)

In [ ]:
total_words = len(tokenizer.word_index)+1
print("Vocabulary Size: ", total_words)

Vocabulary Size:  9608


In [ ]:
input_sequences = []
for line in datastore:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
print(f"Total Number of Sequences: {len(input_sequences)}")

Total Number of Sequences: 44981


In [ ]:
max_length = max([len(line) for line in input_sequences])
print(f"Maximum Length of Sentence: {max_length}")

Maximum Length of Sentence: 306


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_length, padding='pre'))

In [ ]:
xs = input_sequences[:,:-1]
labels = input_sequences[:, -1]

In [ ]:
print(len(xs), len(labels))

44981 44981


In [ ]:
import tensorflow as tf

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(total_words, 240, input_length=max_length-1),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation=tf.nn.relu),
  tf.keras.layers.Dense(total_words, activation=tf.nn.softmax)
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 305, 240)          2305920   
                                                                 
 bidirectional (Bidirectiona  (None, 305, 200)         272800    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               59648     
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 100)               6500      
                                                                 
 dense_1 (Dense)             (None, 9608)              9

In [ ]:
history = model.fit(xs, ys, epochs=100)

Epoch 1/100
1406/1406 [==============================] - 74s 46ms/step - loss: 7.4869 - accuracy: 0.0697
Epoch 2/100
1406/1406 [==============================] - 64s 46ms/step - loss: 6.9783 - accuracy: 0.0827
Epoch 3/100
1406/1406 [==============================] - 65s 46ms/step - loss: 6.6859 - accuracy: 0.1043
Epoch 4/100
1406/1406 [==============================] - 65s 46ms/step - loss: 6.4558 - accuracy: 0.1197
Epoch 5/100
1406/1406 [==============================] - 65s 46ms/step - loss: 6.2465 - accuracy: 0.1289
Epoch 6/100
1406/1406 [==============================] - 65s 46ms/step - loss: 6.0440 - accuracy: 0.1371
Epoch 7/100
1406/1406 [==============================] - 65s 46ms/step - loss: 5.8467 - accuracy: 0.1466
Epoch 8/100
1406/1406 [==============================] - 65s 46ms/step - loss: 5.6548 - accuracy: 0.1551
Epoch 9/100
1406/1406 [==============================] - 65s 46ms/step - loss: 5.4744 - accuracy: 0.1644
Epoch 10/100
1406/1406 [==============================]

In [ ]:
model.save("/content/drive/MyDrive/Models(AI)/next_w_g.h5")